In [1]:
import rdflib
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import FOAF , XSD, Namespace
import pandas as pd
import spacy

### Load Existing RDF Graph

In [2]:
graph = rdflib.Graph()
graph.parse('./data/starwars.ttl', format='turtle')

<Graph identifier=Na487a696bc7545e0ad478a7e994e40c7 (<class 'rdflib.graph.Graph'>)>

### Load New Triples from Text2Graph

In [3]:
triples = pd.read_csv('./data/text2rdf_processed.csv')
triples

,Type1,Entity1,Relationship,Type2,Entity2
0,Node,Anakin Skywalker,is fictional character in,Node,Star Wars
1,Node,Anakin Skywalker,is,Node,fictional character
2,Node,fictional character,is in,Node,Star Wars
3,Node,Anakin Skywalker,appears as,Node,pivotal antagonist
4,Node,Anakin Skywalker,appears in,Node,film trilogy
5,Node,Anakin Skywalker,are,Node,central
6,Node,Anakin Skywalker,are central to,Node,narrative
7,Node,Anakin Skywalker,are central to,Node,narrative of film trilogy
8,Node,Anakin Skywalker,appears as,Node,antagonist
9,Node,actions,drive plot,Node,central to narrative of film trilogy


### Find the 'Anakin Skywalker' URIRef

In [4]:
query_str = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX voc: <https://swapi.co/vocabulary/>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


    SELECT ?s ?o
    WHERE {   
        ?s rdfs:label ?o.
        FILTER(?o='Anakin Skywalker')
    }
"""
res = graph.query(query_str)
list(res)

[(rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.Literal('Anakin Skywalker', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))]

### Get all properties of Anakin Skywalker

In [5]:
query_str = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX voc: <https://swapi.co/vocabulary/>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX human: <https://swapi.co/resource/human/>


    SELECT ?p
    WHERE {   
        ?s ?p ?o.
        FILTER(?s=human:11)
    }
    #LIMIT 10
"""
res = graph.query(query_str)
list(res)

[(rdflib.term.URIRef('https://swapi.co/vocabulary/desc')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/height')),
 (rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/gender')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/starship')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/homeworld')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/starship')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/film')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/starship')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/skinColor')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/mass')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/film')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/eyeColor')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/vehicle')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/film')),
 (rdflib.term.URIRef('https://swapi.co/vocabulary/vehicle')),
 (rdf

### Node to Insert

In [6]:
new_node = triples.iloc[13]
new_node

Type1                       Node
Entity1         Anakin Skywalker
Relationship      was created by
Type2                       Node
Entity2             George Lucas
Name: 13, dtype: object

### Look for George Lucas

In [7]:
query_str = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX voc: <https://swapi.co/vocabulary/>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


    SELECT ?s ?p ?o
    WHERE {   
        ?s ?p ?o.
        FILTER(?o='George Lucas')
    }
"""
res = graph.query(query_str)
list(res)

[(rdflib.term.URIRef('https://swapi.co/resource/film/6'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/director'),
  rdflib.term.Literal('George Lucas', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
 (rdflib.term.URIRef('https://swapi.co/resource/film/4'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/director'),
  rdflib.term.Literal('George Lucas', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
 (rdflib.term.URIRef('https://swapi.co/resource/film/1'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/director'),
  rdflib.term.Literal('George Lucas', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
 (rdflib.term.URIRef('https://swapi.co/resource/film/5'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/director'),
  rdflib.term.Literal('George Lucas', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))]

### Insert to Graph

In [8]:
def create_namespace(namespace, text):
    components = text.lower().split(' ')
    new_text = components[0] + ''.join(x.title() for x in components[1:])
    return Namespace(namespace) + new_text

In [12]:
new_entitynode = create_namespace('https://www.example.com/', 'createdBy')
new_entitynode

'https://www.example.com/createdby'

In [13]:
URIRef(new_entitynode)

rdflib.term.URIRef('https://www.example.com/createdby')

In [14]:
EX = Namespace('https://www.example.com/')
graph.add((URIRef("https://swapi.co/resource/human/11"), URIRef(new_entitynode), Literal(new_node['Entity2'])))

In [16]:
query_str = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX voc: <https://swapi.co/vocabulary/>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX human: <https://swapi.co/resource/human/>


    SELECT ?s ?p ?o
    WHERE {   
        ?s ?p ?o.
        FILTER(?s=human:11)
    }
"""
res = graph.query(query_str)
list(res)

[(rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/starship'),
  rdflib.term.URIRef('https://swapi.co/resource/starship/65')),
 (rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/Human')),
 (rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/vehicle'),
  rdflib.term.URIRef('https://swapi.co/resource/vehicle/44')),
 (rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/eyeColor'),
  rdflib.term.Literal('blue', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))),
 (rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('https://swapi.co/vocabulary/Character')),
 (rdflib.term.

### Entity Linking

Entity linking resolves ambiguous textual references to concepts in a knowledge base

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [19]:
text = "Tennis champion Emerson was expected to win Wimbledon."

doc = nlp(text)
for ent in doc.ents:
    print(f"Named Entity '{ent.text}' with label '{ent.label_}'")

Named Entity 'Emerson' with label 'PERSON'
Named Entity 'Wimbledon' with label 'ORG'


In [25]:
query_str = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX voc: <https://swapi.co/vocabulary/>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


    SELECT ?s ?p ?o
    WHERE {   
        ?s ?p ?o.
        FILTER(?o='Anakin Skywalker')
    }
"""
res = graph.query(query_str)
list(res)

[(rdflib.term.URIRef('https://swapi.co/resource/human/11'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Anakin Skywalker', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))]